In [2]:
%load_ext iminizinc

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime
# import talib
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
from decimal import *
getcontext().prec = 6
import seaborn as sns

# import mplfinance as mpf
%matplotlib inline

mpl.rcParams.update({'font.size': 3, 'lines.linewidth': 0.5, 'figure.dpi': 300})
plt.rcParams['lines.linewidth'] = 0.5 



The iminizinc extension is already loaded. To reload it, use:
  %reload_ext iminizinc


In [ ]:

def plot_candles(wdf, ax=None, kwargs={}):
    if ax is None:
        fig, ax = plt.subplots(**kwargs)

    up, down = wdf[wdf.close >= wdf.open], wdf[wdf.close < wdf.open]
    col1,col2 = 'green','red'
    width, width2 = .3, .07
    # Plotting up prices of the stock
    ax.bar(up.index, up.close-up.open, width, bottom=up.open, color=col1)
    ax.bar(up.index, up.high-up.close, width2, bottom=up.close, color=col1)
    ax.bar(up.index, up.low-up.open, width2, bottom=up.open, color=col1)
    # Plotting down prices of the stock
    ax.bar(down.index, down.close-down.open, width, bottom=down.open, color=col2)
    ax.bar(down.index, down.high-down.open, width2, bottom=down.open, color=col2)
    ax.bar(down.index, down.low-down.close, width2, bottom=down.close, color=col2)    
    plt.xticks(rotation=30, ha='right')

In [ ]:
odf = pd.read_json(r'../freq-user-data/data/binance/BTC_USDT-1d.json').dropna()
odf.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

odf['date'] = pd.to_datetime(odf['timestamp'], unit='ms', utc=False)
odf['idate'] = odf.date.dt.strftime('%Y%m%d')
odf.set_index(pd.DatetimeIndex(odf["date"]), inplace=True, drop=True)
odf = odf.sort_index()
odf

In [ ]:
plot_candles(odf.tail(100),kwargs={'figsize': (9,3)})


## Last Swing Low -> Swing High 

In [ ]:
odf = pd.read_json(r'../freq-user-data/data/binance/BTC_USDT-1d.json').dropna()
odf.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

odf['date'] = pd.to_datetime(odf['timestamp'], unit='ms', utc=False)
odf['idate'] = odf.date.dt.strftime('%Y%m%d')
odf.set_index(pd.DatetimeIndex(odf["date"]), inplace=True, drop=True)
odf = odf.sort_index()
odf

In [ ]:
from minizinc import Instance, Model, Solver
import nest_asyncio
nest_asyncio.apply()

fibs = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]

dlen = len(odf)

# sl_start= IntSlider(description="Str", min=0, max=dlen, step=10, value=dlen-200)
# sl_end = IntSlider(description="End", min=0, max=dlen, value=dlen, step=10)
sl_range = IntRangeSlider(
    value=[int(dlen-1000), dlen],
    min=0,
    max=dlen,
    step=50,
    description='Rng',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)
sl_now = IntSlider(description="Now", min=0, max=dlen, value=dlen, step=50)

# def change_end(val):
#     sl_start.value = sl_end.value - 700
#     sl_end.observe(change_end, 'value')

def range_changed(v):
    wstart, wend = sl_range.value
    sl_now.max = wend-wstart
    # print('Calc wdf.shape: ', wdf.shape)


sl_range.observe(range_changed, 'value')

ui = widgets.VBox([VBox([sl_range, sl_now])])
# wstart, wend

kl_n, kl_close, kl_idate, inow = None,None,None,None
result, ipeak_1, itrough_0, ipeak_0 = None, None, None, None
wdf = None

def printer(wrange, inow):
    global kl_n, kl_close, kl_idate
    global ipeak_1, itrough_0, ipeak_0, result
    global wdf

    wstart, wend = sl_range.value
    wdf = odf.iloc[wstart:wend, :]
    print('inow=',inow)
    if wdf is None: return
    # wdf = odf.iloc[wstart:wend, :]

    kl_n = wdf.shape[0]
    kl_close = wdf.close.astype(int).tolist()
    kl_idate = wdf.idate.to_list()
    
    model_params = {'kl_n': kl_n, 'inow': inow, 'kl_close': kl_close}
    with open('swing_high.json', 'w+') as f: f.write(json.dumps(model_params, indent=0))

    # print('Model params:', model_params)
    swing_high = Model("./swing_high.mzn")
    gecode = Solver.lookup("gecode")
    instance = Instance(gecode, swing_high)

    # instance.branch()
    for k,v in model_params.items(): instance[k] = v
    result = instance.solve()

    print(result)
    i_1, i_2, i_3, obj = [result['i_1'],result['i_2'],result['i_3'],result['obj']]

    swing_low, swing_high = kl_close[i_2-1], kl_close[i_3-1]
    fib_levels = [swing_low + f*(swing_high-swing_low) for f in fibs]
    # %minizinc swing_high kl_n=kl_n
    
    # s = mpf.make_mpf_style(base_mpf_style='charles', rc={'font.size': 8})
    # fig = mpf.figure(figsize=(15, 4), style=s) # pass in the self defined style to the whole canvas
    # ax = fig.add_subplot(1,1,1) # main candle stick chart subplot, you can also pass in the self defined style here only for this subplot
    # av = fig.add_subplot(2,1,2, sharex=ax)  # volume chart subplot
    # wdf.close.plot(ax=ax, lw=1)

    ################################
    # ax.plot(wdf.close, lw=1)
    # ax.axvline(wdf.index[inow-1], c='b', lw=0.7)
    # ax.axvline(wdf.index[i_1-1], c='g', lw=0.7)
    # ax.axvline(wdf.index[i_3-1], c='g', lw=0.7)
    # ax.axvline(wdf.index[i_2-1], c='r', lw=0.7)
    # ax.hlines(fib_levels, wdf.index[i_2-1], wdf.index[inow-1], lw=0.5)

    ################################
    # for f in fibs:
    #     l = swing_low + f*(swing_high-swing_low)
        

    # for v in [ipeak_0, ipeak_1, itrough_0]: ax.axvline(v-1, lw=0.7)
    ####################
    # mpf.plot(wdf, type='candle', ax=ax)

    fig, ax = plt.subplots(figsize=(9,3))
    plot_candles(wdf, ax=ax)
    ax.axvline(inow-1, c='b', lw=0.7)
    # ax.axvline(i_1-1, c='g', lw=0.7)
    # ax.axvline(wdf.index[i_3-1], c='g', lw=0.7)
    # ax.axvline(wdf.index[i_2-1], c='r', lw=0.7)
    # ax.hlines(fib_levels, i_2-1, inow-1, lw=0.5)
    plt.show()

out = widgets.interactive_output(printer, {'wrange': sl_range, 'inow': sl_now})
display(ui, out)


In [ ]:
result

### FInd all swing-low-->swing-highs and fib levels

In [ ]:
from minizinc import Instance, Model, Solver
import nest_asyncio
nest_asyncio.apply()
%matplotlib inline
# %matplotlib

strategy = 'all-swing_highs'
strategy_params_json = f'./strategy-params/{strategy}.json'

fibs = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]

dlen = len(odf)

kl_n, kl_close, kl_idate, inow = None,None,None,None
idxs = None
wdf = None

sl_range = IntRangeSlider(
    value=[int(dlen-1000), dlen],
    min=0,
    max=dlen,
    step=100,
    description='Rng',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

strategy_params = {}
strategy_params = {"wrange": {'wdg': sl_range}, **strategy_params}
wdgts = [pv['wdg'] for pk, pv in strategy_params.items()]

ui = widgets.VBox([widgets.HBox(wdgts[i:i+4]) for i in range(0, len(wdgts), 4)])
# ui = widgets.VBox([VBox([sl_range, sl_now])])
idxs, nsn, lha = None, None, None


if os.path.exists(strategy_params_json):
    with open(strategy_params_json) as f: 
        js = json.loads(f.read());
        for k, v in strategy_params.items(): 
            if k in js: v['wdg'].value=js[k];

def printer(wrange):
    with open(strategy_params_json, "w") as f: f.write(json.dumps({k: v['wdg'].value for k, v in strategy_params.items()}))
    global kl_n, kl_close, kl_idate
    global idxs, nsn, lha
    global wdf
    wstart, wend = sl_range.value

    print('Length: ', wend - wstart + 1)

    wdf = odf.iloc[wstart:wend, :]
    if wdf is None: return
    # wdf = odf.iloc[wstart:wend, :]

    kl_close = wdf.close.astype(int).values
    
    ################################
    ### Call Minizinc solver
    ##------------------------------

    model_params = {'ser_n': len(kl_close), 'series': kl_close.tolist()}
    with open('all-swing_highs.json', 'w+') as f: f.write(json.dumps(model_params, indent=0))

    #######

    # print('Model params:', model_params)

    swing_high = Model("./all-swing_highs.mzn")
    gecode = Solver.lookup("gecode")
    instance = Instance(gecode, swing_high)

    for k,v in model_params.items(): instance[k] = v
    result = instance.solve()
    with open('all-swing_highs-result.json', 'w+') as f: f.write(json.dumps({'lha': result['lha'], 'nsn': result['nsn']}))


    # with open('all-swing_highs-result.json') as f: result = json.loads(f.read())

    lha, nsn = resarr = [result['lha'],result['nsn']]
    print(f'nsn={nsn}\nlha=\n{lha[0]}\n{lha[1]}')


    idxs = (np.array(lha) - 1).T.tolist()
    # ##############################################################
    
    # s = mpf.make_mpf_style(base_mpf_style='charles', rc={'font.size': 8})
    # fig = mpf.figure(figsize=(15, 4), style=s) # pass in the self defined style to the whole canvas
    # plt.clf()
    # plt.rcParams['figure.figsize'] = [9,5]
    fig = plt.figure(figsize=(9, 5))
    ax = fig.add_subplot(1,1,1) 


    ###############################
    plt.plot(wdf.close, lw=0.5)
    for i in range(nsn):
        r = idxs[i]
        x = wdf.index[r]
        y = kl_close[r]
        sw_l, sw_h = kl_close[r[0]], kl_close[r[1]]
        ax.vlines(wdf.index[r], ymin=sw_l, ymax=sw_h, lw=0.7, colors=['gray', 'green', 'red', 'blue'])
        ax.hlines([sw_l, sw_h], xmin=wdf.index[r[0]], xmax=wdf.index[r[1]], lw=0.7, colors='gray')
        ax.plot(x,y, c='g', lw=0.5)
        # plt.plot[]
        # ax.axvline(wdf.index[idxs[i][0]], c='g', lw=0.7)
        # ax.axvline(wdf.index[idxs[i][1]], c='r', lw=0.7)
        # ax.axvline(wdf.index[idxs[i][2]], c='g', lw=0.7)
        # ax.axvline(wdf.index[idxs[i][3]], c='b', lw=0.7)
        # ax.axvline(wdf.index[idxs[i][0]], c='g', lw=0.7)
        # ax.axvline(wdf.index[idxs[i][1]], c='r', lw=0.7)
        # ax.axvline(wdf.index[idxs[i][2]], c='g', lw=0.7)

    # ax.axvline(wdf.index[inow-1], c='b', lw=0.7)
    # ax.axvline(wdf.index[i_1-1], c='g', lw=0.7)
    # ax.axvline(wdf.index[i_3-1], c='g', lw=0.7)
    # ax.axvline(wdf.index[i_2-1], c='r', lw=0.7)
    # ax.hlines(fib_levels, wdf.index[i_2-1], wdf.index[inow-1], lw=0.5)

    ################################
    ################################
    ################################
    # mpf.plot(wdf, type='candle', ax=ax)
    # ax.axvline(inow-1, c='b', lw=0.7)
    # # ax.axvline(i_1-1, c='g', lw=0.7)
    # ax.axvline(i_3-1, c='g', lw=0.7)
    # ax.axvline(i_2-1, c='r', lw=0.7)
    # ax.hlines(fib_levels, i_2-1, inow-1, lw=0.5)

    plt.show();
    

out = widgets.interactive_output(printer, {k : v['wdg'] for k,v in strategy_params.items()});
x = display(ui, out);
plt.close()
# fig

In [ ]:
wdf

## Explore ranges

In [ ]:
odf = pd.read_json(r'../freq-user-data/data/binance/BTC_USDT-4h.json').dropna()
odf.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

odf['date'] = pd.to_datetime(odf['timestamp'], unit='ms', utc=False)
# df.index = df['time']
# df.set_index('time', drop=True, inplace=True)
odf['idate'] = odf.date.dt.strftime('%Y%m%d')
odf.set_index(pd.DatetimeIndex(odf["date"]), inplace=True, drop=True)
# df = df[['time', 'symbol', 'source', 'resolution', 'open', 'high', 'low', 'close', 'volume']]
# df.to_csv (r'./data/binance/BTC_USDT-5m.csv', index = None)
# df.set_index('time')
odf = odf.sort_index()
odf

### Rolling peaks

In [ ]:
wnd = 200
jmp = 50
# stra_params = {'lag': sl_lag.value, 'hold': sl_hold.value, 'pkmin': sl_peaks.value[0], 'pkmax': sl_peaks.value[1]}

idf = pd.DataFrame(index=odf.index)
idf['e'] = odf.close.ewm(span=50).mean()

#sample entries
entries=(odf.close < idf.e) & (odf.close.shift() > idf.e)
hold_periods=[6, 12, 24]


def analyze_trades(odf, entries=[], hold_periods=[]):
    odf_split = [ (i,odf.iloc[i:i+wnd], entries.iloc[i:i+wnd]) for i in [s * jmp for s in range((odf.shape[0] - wnd) // jmp + 1)] ]   
    adf = pd.DataFrame.from_dict(
    {
        **{
            (f'Ho_{ho}', 'L', i) : wdf.close.divide(wdf.low.rolling(ho).min().shift(-ho)).apply(np.log).describe() 
                for (i,wdf, wentries) in odf_split
                for ho in hold_periods
        },
        **{
            (f'Ho_{ho}', 'H', i) : wdf.close.divide(wdf.high.rolling(ho).max().shift(-ho)).apply(np.log).describe() 
                for (i,wdf, wentries) in odf_split
                for ho in hold_periods
        }
    }
    ).unstack().unstack(level=0).unstack(level=0).unstack(level=-1)

    return adf

adf = analyze_trades(odf, entries)

## Resampling to higher timeframes

In [ ]:
odf = pd.read_json(r'../freq-user-data/data/binance/BTC_USDT-1d.json').dropna()
odf.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

odf['date'] = pd.to_datetime(odf['timestamp'], unit='ms', utc=False)
# df.index = df['time']
# df.set_index('time', drop=True, inplace=True)
odf['idate'] = odf.date.dt.strftime('%Y%m%d')
odf.set_index(pd.DatetimeIndex(odf["date"]), inplace=True, drop=True)
# df = df[['time', 'symbol', 'source', 'resolution', 'open', 'high', 'low', 'close', 'volume']]
# df.to_csv (r'./data/binance/BTC_USDT-5m.csv', index = None)
# df.set_index('time')
odf = odf.sort_index()
odf.head(3)

In [ ]:
# Resample and get quoterly
cohlc = ['open','high', 'low', 'close']
fcohlc = [(c,c) for c in cohlc]
ydf = odf.resample('Y', label='left', closed='right').ohlc()[fcohlc].droplevel(axis=1, level=1)
ydf.set_index(ydf.index + pd.DateOffset(1), inplace=True)
qdf = odf.resample('Q', label='left', closed='right').ohlc()[fcohlc].droplevel(axis=1, level=1)
qdf.set_index(qdf.index + pd.DateOffset(1), inplace=True)
mdf = odf.resample('M', label='left', closed='right').ohlc()[fcohlc].droplevel(axis=1, level=1)
mdf.set_index(mdf.index + pd.DateOffset(1), inplace=True)
w2df = odf.resample('2W', label='left', closed='right').ohlc()[fcohlc].droplevel(axis=1, level=1)
w2df.set_index(w2df.index + pd.DateOffset(1), inplace=True)
wdf = odf.resample('W', label='left', closed='right').ohlc()[fcohlc].droplevel(axis=1, level=1)
wdf.set_index(wdf.index + pd.DateOffset(1), inplace=True)
d3df = odf.resample('3D', label='left', closed='right').ohlc()[fcohlc].droplevel(axis=1, level=1)
d3df.set_index(d3df.index + pd.DateOffset(1), inplace=True)
ydf


### Daily pivot levels

# What are Pivot Points?
Pivot points are price levels often used in intraday trading (but they can be used even in swing trading). They are considered “natural” supports and resistances for the price during a daily market session, so they can be quite useful for day traders because the market often behaves in a non-trivial way when it gets closer to them.

There are several types of pivot points, but the classic pivot points are usually defined starting from the last day’s high, low, and close price. Applying some mathematical calculations to these prices on the previous market day, a trader can calculate the pivot levels for the current market day and use them. In this article, I’ll focus on classical pivot points.

A day trader could look for trading opportunities where the price is near a pivot level, waiting for a pullback or a breakout and using the nearest pivot level as a target, for example.

# How to calculate them

- PP = (High + Low + Close) / 3
- R1 = 2 * PP — Low
- S1 = 2 * PP — High
- R2 = PP + (High — Low)
- S2 = PP — (High — Low)
- R3 = PP + 2 * (High — Low)
- S3 = PP — 2 * (High — Low)

# Ideas

- Up or Down fib channels, levels, breaking, rejecting
- Swing low/high fib levels
- 3 recent highs and rejection
- rectangular areas / ranges
- Optimize a strategy for 2/1 risk 2 reward
- Large wicks through support
- Weekly/Monthly/Quarterly support/resistence


## Fish for Risk2Reward

In [ ]:
# Load candles
odf = pd.read_json(r'../freq-user-data/data/binance/BTC_USDT-1d.json').dropna()
odf.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

odf['date'] = pd.to_datetime(odf['timestamp'], unit='ms', utc=False)
# odf['idate'] = odf.date.dt.strftime('%Y%m%d')
odf.set_index(pd.DatetimeIndex(odf["date"]), inplace=True, drop=True)
# odf = odf.sort_index()
odf = odf[['open', 'high', 'low', 'close']]
odf.head(3)

In [ ]:
wdf = odf.iloc[:100]#.set_axis(labels=map(str.capitalize, wdf.columns), axis=1,copy=True)
# wdf.set_axis(labels=map(str.capitalize, wdf.columns), axis=1,copy=True)
wdf
# candles

In [ ]:
# windowed
wnd = 300
jmp = 50
hold_periods=[3, 7, 14]
# def analyze_trades(odf, entries=[], hold_periods=[]):
odf_split = [ (i,odf.iloc[i:i+wnd]) for i in [s * jmp for s in range((odf.shape[0] - wnd) // jmp + 1)] ]   
adf = pd.DataFrame.from_dict(
{
    **{
        (f'Ho_{ho}', 'L', i) : wdf.close.divide(wdf.low.rolling(ho).min().shift(-ho)).apply(np.log).describe() 
            for (i,wdf) in odf_split
            for ho in hold_periods
    },
    **{
        (f'Ho_{ho}', 'H', i) : wdf.close.divide(wdf.high.rolling(ho).max().shift(-ho)).apply(np.log).describe() 
            for (i,wdf) in odf_split
            for ho in hold_periods
    }
}
).unstack().unstack(level=0).unstack(level=0).unstack(level=-1)

# for ho in hold_periods: wdf[(r2r)
#     # return adf
# idx=pd.IndexSlice
adf.loc[idx[:], idx[:,['H', 'L'],'50%']].head(3)
# adf
# adf = analyze_trades(odf, entries)

In [ ]:
### R2R fishing
from matplotlib import pyplot as plt


# plot_candles(wdf)

def add_areas(wdf, hold, r2r):
    wdf['rfd'] = wdf.low.rolling(hold).min().shift(-hold)
    wdf['rfu'] = wdf.high.rolling(hold).max().shift(-hold)
    wdf['prfu'] = np.log(wdf.rfu / wdf.close)
    wdf['prfd'] = np.log(wdf.rfd / wdf.close)
    wdf['pr2r'] = wdf.prfu / - wdf.prfd

    wdf['r2ru_in'] = wdf.pr2r >= r2r
    wdf['r2ru_out'] = ~wdf.r2ru_in
    wdf['r2ru_start'] = wdf.r2ru_in & wdf.r2ru_out.shift()
    wdf['r2ru_end'] = (wdf.r2ru_out & (wdf.index > wdf[wdf.r2ru_start].index[0]) & wdf.r2ru_in.shift()).shift(-1).fillna(False)
    wdf['r2rd_in'] = wdf.pr2r** -1 >= r2r 
    wdf['r2rd_out'] = ~wdf.r2rd_in
    wdf['r2rd_start'] = wdf.r2rd_in & wdf.r2rd_out.shift()
    wdf['r2rd_end'] = (wdf.r2rd_out & (wdf.index >= wdf[wdf.r2rd_start].index[0]) & wdf.r2rd_in.shift()).shift(-1).fillna(False)

strategy = 'r2r-fishing'
strategy_params_json = f'./strategy-params/{strategy}.json'

mpl.rcParams.update({'font.size': 3, 'lines.linewidth': 0.5, 'figure.dpi': 300})
dlen = len(odf)

sl_range = IntRangeSlider(value=[int(dlen-1000), dlen],min=0,max=dlen,step=10,description='Rng',continuous_update=False)
sl_hold = IntSlider(value=30,min=1,max=50,step=1,description='Hold',continuous_update=False)
sl_r2r = FloatSlider(value=2,min=1,max=5,step=0.1,description='R2R',continuous_update=False)
sl_pq = FloatSlider(value=0.5,min=0,max=1,step=0.05,description='PQ',continuous_update=False)
sl_rq = FloatSlider(value=0.5,min=0,max=1,step=0.05,description='RQ',continuous_update=False)
sl_rwnd = IntSlider(value=100,min=10,max=dlen,step=int(dlen / 20),description='RWnd')
chk_candles = Checkbox(value=True, description='Candles', disabled=False)
dd_dir = Dropdown(options=[('Up', 1), ('Down', -1)], value=1, description='Dir',)

strategy_params = {
    "hold": {'wdg': sl_hold},
    "r2r": {'wdg': sl_r2r},
    "rwnd": {'wdg': sl_rwnd},
    "pq": {'wdg': sl_pq},
    "rq": {'wdg': sl_rq},
    "candles": {"wdg": chk_candles},
    "mdir": {"wdg": dd_dir}
}
strategy_params = {
    "wrange": {'wdg': sl_range}, **strategy_params
}
wdgts = [pv['wdg'] for pk, pv in strategy_params.items()]

ui = widgets.VBox([widgets.HBox(wdgts[i:i+4]) for i in range(0, len(wdgts), 4)])

if os.path.exists(strategy_params_json):
    with open(strategy_params_json) as f: 
        js = json.loads(f.read());
        for k, v in strategy_params.items(): 
            if k in js: v['wdg'].value=js[k];

else: print(f'File not found: {strategy_params_json}')

wdf = None
result = None

def printer(
        wrange, hold, r2r, rwnd, pq, rq, candles, mdir
):
    global wdf
    global result
    with open(strategy_params_json, "w") as f: f.write(json.dumps({k: v['wdg'].value for k, v in strategy_params.items()}))
    
    wstart, wend = wrange
    wdf = odf.iloc[wstart:wend, :].copy()

    add_areas(wdf, hold, r2r)
    x_start = wdf[wdf.r2ru_start].index if mdir == 1 else wdf[wdf.r2rd_start].index
    x_end = wdf[wdf.r2ru_end].index if mdir == 1 else wdf[wdf.r2rd_end].index

    # print(f'N={len(wdf)}, Period: {wdf.index[-1] - wdf.index[0]}, Start: {wdf.index[0]}, End: {wdf.index[-1]}, Areas: {len(x_start)}\n')

    plt.close('all')
    fig = plt.figure(figsize=(9,3))

    (axm2, ax1,ax2, ax3) = fig.subplots(4, 1, height_ratios=[1,3,1,1], sharex=True);
    plot_candles(wdf, ax=ax1) if candles else ax1.plot(wdf.close, lw=0.3, c='b')
    wdf.rfu.plot(ax=ax1, lw=0.04, c='g')
    wdf.rfd.plot(ax=ax1, lw=0.04, c='r')
    for ax,lims in [(ax1,[]),(ax2,[]),(ax3,[]), (axm2,[0.5,1])]: 
        if mdir in [0,1]:
            # for s in x_start: ax.axvline(s, *lims, lw=0.3, c='g')
            # for s in x_end: ax.axvline(s, *lims, lw=0.3, c='r')
            for i in range(len(x_start)) : ax.axvspan(x_start[i], x_end[i], color='gray', alpha=0.2)
        if mdir in [-1,0]:
            # for s in x_start: ax.axvline(s, *[x - 0.5 for x in lims], lw=0.3, c='g')
            # for s in x_end: ax.axvline(s, *[x - 0.5 for x in lims], lw=0.3, c='r')
            for i in range(len(x_start)) : ax.axvspan(x_start[i], x_end[i], color='gray', alpha=0.2)

    wdf.prfu.plot(ax=ax2, lw=0.2, c='g')
    wdf.prfd.plot(ax=ax2, lw=0.2, c='r')
    ax2.axhline(0, lw=0.2, linestyle='--')

    wdf.pr2r.clip(2**-2,2**2).plot(ax=axm2, lw=0.05, c='b', logy=False)
    axm2.axhline(1, lw=0.3, c='b')
    axm2.axhline(r2r, lw=0.2, linestyle='--', c='g')
    axm2.axhline(r2r**-1, lw=0.2, linestyle='--', c='r')

out = widgets.interactive_output(printer, {k : v['wdg'] for k,v in strategy_params.items()});
x = display(ui, out);





## Wick Statistics

In [ ]:
# Load candles
odf = pd.read_json(r'../freq-user-data/data/binance/BTC_USDT-4h.json').dropna()
odf.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

odf['date'] = pd.to_datetime(odf['timestamp'], unit='ms', utc=False)
# odf['idate'] = odf.date.dt.strftime('%Y%m%d')
odf.set_index(pd.DatetimeIndex(odf["date"]), inplace=True, drop=True)
# odf = odf.sort_index()
odf = odf[['open', 'high', 'low', 'close']]
odf

In [ ]:

def add_indicators(wdf, hold):
    wdf['rfd'] = wdf.low.rolling(hold).min().shift(-hold)
    wdf['rfu'] = wdf.high.rolling(hold).max().shift(-hold)
    wdf['lr_fu'] = np.log(wdf.rfu / wdf.close)
    wdf['lr_fd'] = np.log(wdf.rfd / wdf.close)
    # wdf['pr2r'] = wdf.lretfu / - wdf.lretfd

# def add_body_stuff(wdf):
    wdf['bou'] = np.maximum(wdf['open'], wdf['close'])
    wdf['bod'] = np.minimum(wdf['open'], wdf['close'])
    wdf['lr_wiu'] = np.log(wdf['high']/wdf['bou'])
    wdf['lr_wid'] = np.log(wdf['low']/wdf['bod'])


    wdf=wdf.join(wdf[['lr_fu', 'lr_fd', 'lr_wiu', 'lr_wid']].apply(lambda x: x.multiply(100).round(2)).add_prefix('p'))
    # wdf=wdf.join(wdf[['plr_fu', 'plr_fd', 'plr_wiu', 'plr_wid']].apply(lambda x: pd.qcut(x,ncuts)).add_prefix('q'))

    # wdf['qmprfu'] = pd.qcut(wdf.mprfu, 10)
    return wdf


In [ ]:
### Wicks stats
from matplotlib import pyplot as plt
from matplotlib import rcParams
from statsmodels.distributions.empirical_distribution import ECDF, ECDFDiscrete
from scipy.stats import gaussian_kde
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

strategy = 'stats-wicks-analyser'
strategy_params_json = f'./strategy-params/{strategy}.json'

mpl.rcParams.update({'font.size': 3, 'lines.linewidth': 0.5, 'figure.dpi': 300})
dlen = len(odf)

sl_range = IntRangeSlider(value=[int(dlen-1000), dlen],min=0,max=dlen,step=10,description='Rng',continuous_update=False)
sl_hold = IntSlider(value=30,min=1,max=50,step=1,description='Hold',continuous_update=False)
# sl_r2r = FloatSlider(value=2,min=1,max=5,step=0.1,description='R2R',continuous_update=False)
# sl_pq = FloatSlider(value=0.5,min=0,max=1,step=0.05,description='PQ',continuous_update=False)
# sl_thre = FloatText(value=0.01,min=0,max=0.1,step=0.001,description='Thre',continuous_update=False)
# sl_rq = FloatSlider(value=0.5,min=0,max=1,step=0.05,description='RQ',continuous_update=False)
# sl_rwnd = IntSlider(value=100,min=10,max=dlen,step=int(dlen / 20),description='RWnd')
# chk_candles = Checkbox(value=True, description='Candles', disabled=False)
tx_qbins = IntText(value=10,min=1,max=20,step=2,description='qbins')
dd_dir = Dropdown(options=[('Up', 1), ('Down', -1)], value=1, description='Dir',)

strategy_params = {
    "hold": {'wdg': sl_hold},
    # "r2r": {'wdg': sl_r2r},
    # "rwnd": {'wdg': sl_rwnd},
    # "pq": {'wdg': sl_pq},
    # "rq": {'wdg': sl_rq},
    # "candles": {"wdg": chk_candles},
    "mdir": {"wdg": dd_dir},
    "qbins": {"wdg": tx_qbins},
    # "thre": {"wdg": sl_thre}
}
strategy_params = {
    "wrange": {'wdg': sl_range}, **strategy_params
}
wdgts = [pv['wdg'] for pk, pv in strategy_params.items()]

ui = widgets.VBox([widgets.HBox(wdgts[i:i+4]) for i in range(0, len(wdgts), 4)])

if os.path.exists(strategy_params_json):
    with open(strategy_params_json) as f: 
        js = json.loads(f.read());
        for k, v in strategy_params.items(): 
            if k in js: v['wdg'].value=js[k];

else: print(f'File not found: {strategy_params_json}')

wdf = None
result = None
ax1, ax2 = None, None

def printer(
        wrange, hold, mdir, qbins
):
    global wdf
    global ax1, ax2 
    with open(strategy_params_json, "w") as f: f.write(json.dumps({k: v['wdg'].value for k, v in strategy_params.items()}))
    
    wstart, wend = wrange
    wdf = odf.iloc[wstart:wend, :].copy()

    wdf = add_indicators(wdf, hold)
    # qbins = np.linspace(0,1,qbins+1)
    df_quants = wdf[['plr_fu','plr_fd','plr_wiu','plr_wid']].apply(lambda x: x.abs().quantile(np.linspace(0,1,qbins+1)))
    wdf = wdf.dropna()
    # x_start = wdf[wdf.r2ru_start].index if mdir == 1 else wdf[wdf.r2rd_start].index
    # x_end = wdf[wdf.r2ru_end].index if mdir == 1 else wdf[wdf.r2rd_end].index

    print(f'N={len(wdf)}, Period: {wdf.index[-1] - wdf.index[0]}, Start: {wdf.index[0]}, End: {wdf.index[-1]}\n')

    # if mdir == -1: entries = wdf[wdf.rehb>=thre].index  
    # else: entries = wdf[wdf.relb<=-thre].index
    # print(f'mdir:{mdir}, thre: {thre}, entries: {len(entries)}\n')

    # edf = wdf.loc[entries]

    fig2 = plt.figure(layout='constrained', figsize=(9,4))
    # fig = plt.figure(figsize=(9,5))
    # fig1,fig2 = fig.subfigures(2,1, wspace=0.001, hspace=0, height_ratios=[1,1])

    [axr1, axr2] = fig2.subplots(2,4, width_ratios=[1,1,1,1], subplot_kw={});
    df_quants.plr_fu.plot(ax=axr1[0], c='g')
    df_quants.plr_fd.plot(ax=axr1[0], c='r')
    axr1[0].set_ylim([0,5])
    axr1[0].axvline(0.5)
    axr1[0].axhline(df_quants.plr_fu[0.5], c='g', linestyle='--')
    axr1[0].axhline(df_quants.plr_fd[0.5], c='r', linestyle='--')
    # df_quants.multiply(-1).plr_fd.plot(ax=axr1[0], c='r')


    plt.show()

out = widgets.interactive_output(printer, {k : v['wdg'] for k,v in strategy_params.items()});
x = display(ui, out);






## Return statistics for entries per period, timeframe, coin

In [ ]:
def load_candles(pair='BTC_USDT', timeframe='1d'):
    odf = pd.read_json(f'../freq-user-data/data/binance/{pair}-{timeframe}.json').dropna()
    odf.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

    odf['date'] = pd.to_datetime(odf['timestamp'], unit='ms', utc=False)
    # odf['idate'] = odf.date.dt.strftime('%Y%m%d')
    odf.set_index(pd.DatetimeIndex(odf["date"]), inplace=True, drop=True)
    # odf = odf.sort_index()
    odf = odf[['open', 'high', 'low', 'close']]
    return odf

In [ ]:
def add_hold_return(wdf, hold):
    wdf['rfd'] = wdf.low.rolling(hold).min().shift(-hold)
    wdf['rfu'] = wdf.high.rolling(hold).max().shift(-hold)
    wdf['lr_fu'] = np.log(wdf.rfu / wdf.close)
    wdf['lr_fd'] = np.log(wdf.rfd / wdf.close)
    # wdf['pr2r'] = wdf.lretfu / - wdf.lretfd

# def add_body_stuff(wdf):
    # wdf['bou'] = np.maximum(wdf['open'], wdf['close'])
    # wdf['bod'] = np.minimum(wdf['open'], wdf['close'])
    # wdf['lr_wiu'] = np.log(wdf['high']/wdf['bou'])
    # wdf['lr_wid'] = np.log(wdf['low']/wdf['bod'])


    wdf=wdf.join(wdf[['lr_fu', 'lr_fd',]].apply(lambda x: x.multiply(100).round(2)).add_prefix('p'))
    # wdf=wdf.join(wdf[['plr_fu', 'plr_fd', 'plr_wiu', 'plr_wid']].apply(lambda x: pd.qcut(x,ncuts)).add_prefix('q'))

    # wdf['qmprfu'] = pd.qcut(wdf.mprfu, 10)
    return wdf

In [ ]:
assets = ['BTC_USDT', 'ADA_USDT']
timeframes = ['1d', '8h']
hold_periods=[6, 12, 24]

In [ ]:
odf.shape

In [ ]:
## Sample entries

# odf = load_candles()
# wnd = 1
ho = 7
wnd = ho 
jmp = wnd // 2
print(f'ho: {ho}, wnd: {wnd}, jmp: {jmp}')
# jmp
[s*wnd for s in range((odf.shape[0] - wnd) // jmp + 1)][-1] + wnd
# odf.unstack().unstack()

In [ ]:
### Return stats
from matplotlib import pyplot as plt
from matplotlib import rcParams
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
mpl.rcParams.update({'font.size': 3, 'lines.linewidth': 0.5, 'figure.dpi': 300})

assets = ['BTC_USDT', 'ADA_USDT']
timeframes = ['1d', '8h']
hold_periods=[6, 12, 24]

# pd.concat(
[
    [ odf.iloc[i:i+ho] for i in [s * ho for s in range((odf.shape[0] - wnd) // jmp + 1)] ]
    for odf in
    [
        load_candles(asset) for asset in assets
    ]
    for ho in hold_periods
]

# , la)
# def analyze_trades(odf, entries=[], hold_periods=[]):
#     odf_split = [ (i,odf.iloc[i:i+wnd], entries.iloc[i:i+wnd]) for i in [s * jmp for s in range((odf.shape[0] - wnd) // jmp + 1)] ]   
#     adf = pd.DataFrame.from_dict(
#     {
#         **{
#             (f'Ho_{ho}', 'L', i) : wdf.close.divide(wdf.low.rolling(ho).min().shift(-ho)).apply(np.log).describe() 
#                 for (i,wdf, wentries) in odf_split
#                 for ho in hold_periods
#         },
#         **{
#             (f'Ho_{ho}', 'H', i) : wdf.close.divide(wdf.high.rolling(ho).max().shift(-ho)).apply(np.log).describe() 
#                 for (i,wdf, wentries) in odf_split
#                 for ho in hold_periods
#         }
#     }
#     ).unstack().unstack(level=0).unstack(level=0).unstack(level=-1)

#     return adf



In [ ]:
qbins = np.linspace(0,1,11)
quantiles = wdf[['plr_fu','plr_fd','plr_wiu','plr_wid']].apply(lambda x: x.abs().quantile(qbins))
quantiles
# wdf['cmprwid'] = 
# d = pd.cut(wdf.mprfu, bins=wdf.mprwid.quantile(qbins).values)
# d[~d.isna()]
# wdf.mprfu
# wdf.mprwid.quantile(np.linspace(0,1,11)).values
# pd.qcut(wdf.mprfu, 10)

In [ ]:
### Wicks stats
from matplotlib import pyplot as plt
from statsmodels.distributions.empirical_distribution import ECDF, ECDFDiscrete

strategy = 'stats-wicks-analyser'
strategy_params_json = f'./strategy-params/{strategy}.json'

mpl.rcParams.update({'font.size': 3, 'lines.linewidth': 0.5, 'figure.dpi': 300})
dlen = len(odf)

sl_range = IntRangeSlider(value=[int(dlen-1000), dlen],min=0,max=dlen,step=10,description='Rng',continuous_update=False)
sl_hold = IntSlider(value=30,min=1,max=50,step=1,description='Hold',continuous_update=False)
# sl_r2r = FloatSlider(value=2,min=1,max=5,step=0.1,description='R2R',continuous_update=False)
# sl_pq = FloatSlider(value=0.5,min=0,max=1,step=0.05,description='PQ',continuous_update=False)
# sl_thre = FloatText(value=0.01,min=0,max=0.1,step=0.001,description='Thre',continuous_update=False)
# sl_rq = FloatSlider(value=0.5,min=0,max=1,step=0.05,description='RQ',continuous_update=False)
# sl_rwnd = IntSlider(value=100,min=10,max=dlen,step=int(dlen / 20),description='RWnd')
# chk_candles = Checkbox(value=True, description='Candles', disabled=False)
dd_dir = Dropdown(options=[('Up', 1), ('Down', -1)], value=1, description='Dir',)

strategy_params = {
    "hold": {'wdg': sl_hold},
    # "r2r": {'wdg': sl_r2r},
    # "rwnd": {'wdg': sl_rwnd},
    # "pq": {'wdg': sl_pq},
    # "rq": {'wdg': sl_rq},
    # "candles": {"wdg": chk_candles},
    "mdir": {"wdg": dd_dir},
    # "thre": {"wdg": sl_thre}
}
strategy_params = {
    "wrange": {'wdg': sl_range}, **strategy_params
}
wdgts = [pv['wdg'] for pk, pv in strategy_params.items()]

ui = widgets.VBox([widgets.HBox(wdgts[i:i+4]) for i in range(0, len(wdgts), 4)])

if os.path.exists(strategy_params_json):
    with open(strategy_params_json) as f: 
        js = json.loads(f.read());
        for k, v in strategy_params.items(): 
            if k in js: v['wdg'].value=js[k];

else: print(f'File not found: {strategy_params_json}')

wdf = None
result = None
entries, edf = [], None

def printer(
        wrange, hold, r2r, rwnd, pq, rq, candles, mdir, thre
):
    global wdf, edf
    global result, entries
    with open(strategy_params_json, "w") as f: f.write(json.dumps({k: v['wdg'].value for k, v in strategy_params.items()}))
    
    wstart, wend = wrange
    wdf = odf.iloc[wstart:wend, :].copy()

    add_body_stuff(wdf)
    # add_areas(wdf, hold, r2r)
    # x_start = wdf[wdf.r2ru_start].index if mdir == 1 else wdf[wdf.r2rd_start].index
    # x_end = wdf[wdf.r2ru_end].index if mdir == 1 else wdf[wdf.r2rd_end].index

    print(f'N={len(wdf)}, Period: {wdf.index[-1] - wdf.index[0]}, Start: {wdf.index[0]}, End: {wdf.index[-1]}\n')

    # if mdir == -1: entries = wdf[wdf.rehb>=thre].index  
    # else: entries = wdf[wdf.relb<=-thre].index
    # print(f'mdir:{mdir}, thre: {thre}, entries: {len(entries)}\n')

    # edf = wdf.loc[entries]

    fig = plt.figure(layout='constrained', figsize=(9,3))
    fig1,fig2 = fig.subfigures(1, 2, wspace=0.001, hspace=0, width_ratios=[3,1])

# axsLeft = subfigs[0].subplots(1, 2, sharey=True)

    (ax1,ax2, ax3) = fig1.subplots(3, 1, height_ratios=[1,1,1], sharex=True);
    # if candles: plot_candles(wdf, ax=ax1)
    # else: ax1.plot(wdf.close, lw=0.3, c='b')

    # ax1.plot(wdf.rfu, lw=0.1, c='g')
    # ax1.plot(wdf.rfd, lw=0.1, c='r')
    # for e in entries: ax1.axvline(e, lw=0.1)
    # for ax,lims in [(ax1,[]),(ax2,[]),(ax3,[]), (axm2,[0.5,1])]: 
    #     if mdir in [0,1]:
    #         # for s in x_start: ax.axvline(s, *lims, lw=0.3, c='g')
    #         # for s in x_end: ax.axvline(s, *lims, lw=0.3, c='r')
    #         for i in range(len(x_start)) : ax.axvspan(x_start[i], x_end[i], color='gray', alpha=0.2)
    #     if mdir in [-1,0]:
    #         # for s in x_start: ax.axvline(s, *[x - 0.5 for x in lims], lw=0.3, c='g')
    #         # for s in x_end: ax.axvline(s, *[x - 0.5 for x in lims], lw=0.3, c='r')
    #         for i in range(len(x_start)) : ax.axvspan(x_start[i], x_end[i], color='gray', alpha=0.2)

    # wdf.prfu.plot(ax=ax2, lw=0.2, c='g')
    # wdf.prfd.plot(ax=ax2, lw=0.2, c='r')
    # ax2.axhline(0, lw=0.2, linestyle='--')

    # wdf.pr2r.clip(2**-2,2**2).plot(ax=axm2, lw=0.05, c='b', logy=True)
    # axm2.axhline(1, lw=0.3, c='b')
    # axm2.axhline(r2r, lw=0.2, linestyle='--', c='g')
    # axm2.axhline(r2r**-1, lw=0.2, linestyle='--', c='r')

    # (rax1,rax2, rax3) = fig2.subplots(3, 1, height_ratios=[1,1,1], sharex=False);
    # rax
    sns.kdeplot(edf.prfu, ax=rax1, color='g', clip=(-0.05,0.1))
    sns.kdeplot(edf.prfd, ax=rax1, color='r', clip=(-0.05,0.1))
    rax2.axhline(0.5)
    # sns.kdeplot(edf.prfu, ax=rax2, color='g', cumulative=True, clip=(-0.05,0.1))
    sns.ecdfplot(edf.prfd, ax=rax2, color='r', clip=(-0.05,0.1))
    rax2.plot(ECDF(edf.prfd.values).x)
    # sns.kdeplot(-edf.prfd, ax=rax2, color='gray', cumulative=True, clip=(-0.05,0.1))
    # rax2.axvline(edf.prfu.quantile(0.5), color='g',)
    rax2.axvline(edf.prfd.quantile(0.6), color='r',)
    
    # rax2.axvline(-edf.prfd.quantile(0.5), color='gray',)

out = widgets.interactive_output(printer, {k : v['wdg'] for k,v in strategy_params.items()});
x = display(ui, out);






## Ranges

In [ ]:
odf = load_candles('BTC_USDT', '4h')

In [87]:

### Ranges
from matplotlib import pyplot as plt


strategy = 'stats-ranges'
strategy_params_json = f'./strategy-params/{strategy}.json'

mpl.rcParams.update({'font.size': 3, 'lines.linewidth': 0.5, 'figure.dpi': 300})

dlen = len(odf)


sl_n2= IntSlider(description="n2", min=0, max=ceil(log(dlen, 2.0)+1), step=1, value=9)
sl_w = IntSlider(description="w", min=0, max=ceil(dlen/pow(2,sl_n2.value-1)-1), step=1, value=1)
def update_sl_w_range(*args):
    sl_w.max = ceil(dlen/pow(2,sl_n2.value-1)-1)
sl_n2.observe(update_sl_w_range, 'value')


# sl_range = IntRangeSlider(value=[int(dlen-1000), dlen],min=0,max=dlen,step=10,description='Rng',continuous_update=False)
sl_hold = IntSlider(value=7,min=1,max=50,step=1,description='Hold',continuous_update=False)
chk_candles = Checkbox(value=True, description='Candles', disabled=False)
dd_dir = Dropdown(options=[('Up', 1), ('Down', -1)], value=1, description='Dir',)

strategy_params = {
    "hold": {'wdg': sl_hold},
    "candles": {"wdg": chk_candles},
    "mdir": {"wdg": dd_dir}
}
# strategy_params = {
#     # "wrange": {'wdg': sl_range}, **strategy_params
# }
wdgts = [pv['wdg'] for pk, pv in strategy_params.items()]

ui = widgets.VBox([
    widgets.HBox([sl_n2, sl_w]),
    widgets.VBox([widgets.HBox(wdgts[i:i+4]) for i in range(0, len(wdgts), 4)])
])

# ui = widgets.VBox([widgets.HBox(wdgts[i:i+4]) for i in range(0, len(wdgts), 4)])

if os.path.exists(strategy_params_json):
    with open(strategy_params_json) as f: 
        js = json.loads(f.read());
        for k, v in strategy_params.items(): 
            if k in js: v['wdg'].value=js[k];

else: print(f'File not found: {strategy_params_json}')

wdf = None
result = None

def printer(
        w2log, w, hold, candles, mdir
):
    global wdf
    global result
    with open(strategy_params_json, "w") as f: f.write(json.dumps({k: v['wdg'].value for k, v in strategy_params.items()}))
    
    
    wsz = floor(pow(2,w2log))
    wst = floor(w * wsz / 2)
    wed = floor(wst + wsz)
    
    wdf = odf.iloc[wst:wed,:].copy()    
    wlen = len(wdf)

    print(f'N={len(wdf)}, Period: {wdf.index[-1] - wdf.index[0]}, Start: {wdf.index[0]}, End: {wdf.index[-1]}\n')

    plt.close('all')
    fig = plt.figure(figsize=(9,3))

    (ax1,ax2, ax3) = fig.subplots(3, 1, height_ratios=[3,1,1], sharex=True);
    plot_candles(wdf, ax=ax1) if candles else ax1.plot(wdf.close, lw=0.3, c='b')

out = widgets.interactive_output(printer, {
        "w2log":sl_n2, "w":sl_w, 
        **{k : v['wdg'] for k,v in strategy_params.items()}
    });
x = display(ui, out);


Output()

In [ ]:
### Ranges
from matplotlib import pyplot as plt


strategy = 'stats-ranges'
strategy_params_json = f'./strategy-params/{strategy}.json'

mpl.rcParams.update({'font.size': 3, 'lines.linewidth': 0.5, 'figure.dpi': 300})
dlen = len(odf)

sl_range = IntRangeSlider(value=[int(dlen-1000), dlen],min=0,max=dlen,step=10,description='Rng',continuous_update=False)
sl_hold = IntSlider(value=7,min=1,max=50,step=1,description='Hold',continuous_update=False)
# sl_r2r = FloatSlider(value=2,min=1,max=5,step=0.1,description='R2R',continuous_update=False)
# sl_pq = FloatSlider(value=0.5,min=0,max=1,step=0.05,description='PQ',continuous_update=False)
# sl_rq = FloatSlider(value=0.5,min=0,max=1,step=0.05,description='RQ',continuous_update=False)
# sl_rwnd = IntSlider(value=100,min=10,max=dlen,step=int(dlen / 20),description='RWnd')
chk_candles = Checkbox(value=True, description='Candles', disabled=False)
dd_dir = Dropdown(options=[('Up', 1), ('Down', -1)], value=1, description='Dir',)

strategy_params = {
    "hold": {'wdg': sl_hold},
    # "r2r": {'wdg': sl_r2r},
    # "rwnd": {'wdg': sl_rwnd},
    # "pq": {'wdg': sl_pq},
    # "rq": {'wdg': sl_rq},
    "candles": {"wdg": chk_candles},
    "mdir": {"wdg": dd_dir}
}
strategy_params = {
    "wrange": {'wdg': sl_range}, **strategy_params
}
wdgts = [pv['wdg'] for pk, pv in strategy_params.items()]

ui = widgets.VBox([widgets.HBox(wdgts[i:i+4]) for i in range(0, len(wdgts), 4)])

if os.path.exists(strategy_params_json):
    with open(strategy_params_json) as f: 
        js = json.loads(f.read());
        for k, v in strategy_params.items(): 
            if k in js: v['wdg'].value=js[k];

else: print(f'File not found: {strategy_params_json}')

wdf = None
result = None

def printer(
        wrange, hold, candles, mdir
):
    global wdf
    global result
    with open(strategy_params_json, "w") as f: f.write(json.dumps({k: v['wdg'].value for k, v in strategy_params.items()}))
    
    wstart, wend = wrange
    wdf = odf.iloc[wstart:wend, :].copy()

    print(f'N={len(wdf)}, Period: {wdf.index[-1] - wdf.index[0]}, Start: {wdf.index[0]}, End: {wdf.index[-1]}\n')

    plt.close('all')
    fig = plt.figure(figsize=(9,3))

    (ax1,ax2, ax3) = fig.subplots(3, 1, height_ratios=[3,1,1], sharex=True);
    plot_candles(wdf, ax=ax1) if candles else ax1.plot(wdf.close, lw=0.3, c='b')

out = widgets.interactive_output(printer, {k : v['wdg'] for k,v in strategy_params.items()});
x = display(ui, out);



